In [1]:
import re
import unicodedata
import numpy as np
from stop_words import get_stop_words
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from collections import Counter

stemmer = FrenchStemmer()
stop_words = get_stop_words('fr')
stop_words = [e.upper() for e in stop_words]
stop_words.append("BR")


def tokenize(text):

    global stop_words
    text = text.replace("'", "")
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    pattern2 = re.compile('www(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = pattern2.sub('', text)
    text = text.replace("(", "").replace(")","")
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore")
    text = text.decode("utf-8")
    tokens = re.compile('\w+').findall(text.upper())
    tokens = [stemmer.stem(e) for e in tokens if ((not (e in stop_words)) and (len(e)>2) and (e[0].isalpha()))]

    return tokens

def tokenize2(text):

    global stop_words
    if type(text) == float:
        print(text)
    text = text.replace("/", " ")
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore")
    text = text.decode("utf-8")
    tokens = re.compile('\w+').findall(text.upper())
    tokens = [stemmer.stem(e) for e in tokens if ((not (e in stop_words)) and (len(e)>2) and (e[0].isalpha()))]

    return tokens





In [ ]:
1+1

In [3]:
nb_offres = 1000

df = pd.read_csv('base_offres.csv', header=0, encoding="utf8")

In [4]:
# Nettoyage df
index = list(df[df['sect'].isnull()].index)
for i in index:
    df.loc[i,'sect'] = df.loc[i,'sect_code']
df.dropna(subset=["sect"], inplace=True)

In [ ]:
dflat = pd.read_csv('regions.csv', header=0, encoding="utf8", sep=';', dtype={"codes_postaux" : str})
dflat.index = dflat.nom_commune
lon = dflat.loc['Lille','longitude']
lon[0]

In [ ]:
dflat = pd.read_csv('regions.csv', header=0, encoding="utf8", sep=';', dtype={"codes_postaux" : str})

dflat.index = dflat.codes_postaux

index = list(df[df['longitude'].isnull() | (df['longitude'] == 0.0)].index)
print(len(index))
df['codes_postaux'] = df['commune'].apply(lambda x: x.split()[-1])

for i in index:
    try:
        df.loc[i,'longitude'] = dflat.loc[df.loc[i,'codes_postaux'],'longitude']
        df.loc[i,'latitude'] = dflat.loc[df.loc[i,'codes_postaux'],'latitude']
    except:
        pass


index = list(df[df['longitude'].isnull() | (df['longitude'] == 0.0)].index)
print(len(index))

def clean_ville(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore")
    text = text.decode("utf-8")
    return text.upper()
    
dflat['nom_commune'] = dflat['nom_commune'].apply(clean_ville)
dflat.index = dflat.nom_commune

for i in index:
    try:
        df.ix[i,'longitude'] = dflat.loc[df.loc[i,'commune_clean'],'longitude']
        df.ix[i,'latitude'] = dflat.loc[df.loc[i,'commune_clean'],'latitude']
    except:
        try:
            df.loc[i,'longitude'] = list(dflat.loc[df.loc[i,'commune_clean'],'longitude'])[0]
            df.loc[i,'latitude'] = list(dflat.loc[df.loc[i,'commune_clean'],'latitude'])[0]
        except:
            pass



In [ ]:
df

In [5]:
df[df["sect"] == 'Services'].index

Int64Index([    0,     1,     3,     7,    14,    25,    33,    39,    40,
               42,
            ...
            59301, 59302, 59303, 59305, 59311, 59312, 59313, 59316, 59319,
            59321],
           dtype='int64', length=8675)

In [6]:
df.drop(df.columns[[0]], axis=1, inplace=True)
df['formation'] + ' ' + df['sect']

0                         Agent de maîtrise/Bac+3 Services
1                                    Sans diplôme Services
2                                                      NaN
3                  Employé/Ouvrier spécialisé/Bac Services
4                                BEP/CAP Textile/Mode/Luxe
5                                                      NaN
6                                                      NaN
7                         Agent de maîtrise/Bac+3 Services
8                                                      NaN
9                                                      NaN
10                                                     NaN
11                           BEP/CAP Commerce/Distribution
12                                                     NaN
13                                                     NaN
14                 Employé/Ouvrier spécialisé/Bac Services
15                                Sans diplôme Agriculture
16                                                     N

In [7]:
df.to_csv('base_offres_propre.csv', encoding='utf8', index=False)

# Base propre 

In [11]:
df = pd.read_csv('base_offres_propre.csv', header=0, encoding="utf8", dtype={"sect": str, "longitude": str, "latitude": str} )
df

,id,date,siren,desc,ref,commune,formation,sect,exp,salaire,...,commune_dep,commune_clean_,commune_clean,nomcommune,longitude,latitude,numrodpartement,desc_,salaire_temp,date_clean
0,ObjectId(585d58c917501629f7b7a475),23 décembre à 17:44,412481301,Nous cherchons le responsable financier &amp; ...,1612cu,Nice 06000,Agent de maîtrise/Bac+3,Services,5 ans et plus,2 000 €,...,6,NICE,NICE,NICE,7.25,43.7,6.0,NOUS CHERCHONS LE RESPONSABLE FINANCIER &AMP; ...,NaN,2016-12-23
1,ObjectId(585d58c917501629f7b7a476),23 décembre à 17:35,433822053,SOCIETE AMBIANCE RECRUTE UN AGENT DE PR...,NaN,Belfort 90000,Sans diplôme,Services,0 à 2 ans,NaN,...,90,BELFORT,BELFORT,BELFORT,6.866667,47.633333,90.0,SOCIETE AMBIANCE RECRUTE UN AGENT DE PR...,9.94,2016-12-23
2,ObjectId(585d58c917501629f7b7a477),23 décembre à 17:35,479482036,Nous sommes une petite boutique spécialisée da...,NaN,Saint-Brieuc 22000,NaN,Commerce/Distribution,0 à 2 ans,1 500 €,...,22,BRIEUC,BRIEUC,BRIEUC,-2.7833330000000003,48.516667,22.0,NOUS SOMMES UNE PETITE BOUTIQUE SPÉCIALISÉE DA...,NaN,2016-12-23
3,ObjectId(585d58c917501629f7b7a478),23 décembre à 17:32,702011172,Securitas Accueil recherche un (e) hôte (sse) ...,LBCCaluire,Caluire-et-Cuire 69300,Employé/Ouvrier spécialisé/Bac,Services,0 à 2 ans,973 €,...,69,CALUIRE-ET-CUIRE,CALUIRE-ET-CUIRE,CALUIRE-ET-CUIRE,4.85,45.8,69.0,SECURITAS ACCUEIL RECHERCHE UN (E) HÔTE (SSE) ...,NaN,2016-12-23
4,ObjectId(585d58c917501629f7b7a479),23 décembre à 17:34,343794731,Découvrez un métier :<br>Opératrice/opérateur ...,NaN,Saint-Jean-le-Blanc 45650,BEP/CAP,Textile/Mode/Luxe,0 à 2 ans,1 467 €,...,45,JEAN-LE-BLANC,JEAN-LE-BLANC,JEAN-LE-BLANC,1.916667,47.9,45.0,DÉCOUVREZ UN MÉTIER :<BR>OPÉRATRICE/OPÉRATEUR ...,NaN,2016-12-23
5,ObjectId(585d58c917501629f7b7a47a),23 décembre à 17:33,819853110,Stéphane Plaza Immobilier Bourgoin-Jallieu<br>...,NaN,Bourgoin-Jallieu 38300,NaN,Immobilier,0 à 2 ans,NaN,...,38,BOURGOIN-JALLIEU,BOURGOIN-JALLIEU,BOURGOIN-JALLIEU,59.0,45.0,38.0,STÉPHANE PLAZA IMMOBILIER BOURGOIN-JALLIEU<BR>...,NaN,2016-12-23
6,ObjectId(585d58c917501629f7b7a47b),23 décembre à 17:33,400863510,La concession Mercedes de Sens (89) recherche ...,NaN,Sens 89100,NaN,Commerce/Distribution,NaN,NaN,...,89,SENS,SENS,SENS,2.0,48.0,89.0,LA CONCESSION MERCEDES DE SENS (89) RECHERCHE ...,NaN,2016-12-23
7,ObjectId(585d58c917501629f7b7a47c),23 décembre à 17:31,489633073,Nous sommes la société Orhès Racing.<br>Nous r...,NaN,Saint-Ouen-l'Aumône 95310,Agent de maîtrise/Bac+3,Services,5 ans et plus,NaN,...,95,OUEN-L-AUMONE,OUEN-L-AUMONE,OUEN-L-AUMONE,2.1166669999999996,49.05,95.0,NOUS SOMMES LA SOCIÉTÉ ORHÈS RACING.<BR>NOUS R...,NaN,2016-12-23
8,ObjectId(585d58ca17501629f7b7a47d),23 décembre à 17:33,528727456,"L’entreprise Coutarel, spécialiste de l’agence...",cacaca,Saint-Domineuc 35190,NaN,Industrie/Environnement,2 à 5 ans,34 500 €,...,35,DOMINEUC,DOMINEUC,DOMINEUC,-1.8666669999999999,48.366667,35.0,"L’ENTREPRISE COUTAREL, SPÉCIALISTE DE L’AGENCE...",NaN,2016-12-23
9,ObjectId(585d58ca17501629f7b7a47e),23 décembre à 17:37,501806657,Dans le cadre de son développement BT CONCEPT ...,NaN,Laval 53000,NaN,BTP/Construction,NaN,NaN,...,53,LAVAL,LAVAL,LAVAL,-0.766667,48.066666999999995,53.0,DANS LE CADRE DE SON DÉVELOPPEMENT BT CONCEPT ...,NaN,2016-12-23


In [12]:
def supprimen(st):
    var = st
    if var[-1:] == '\n':
        return var[:-1]
    else:
        return var
df.dropna(subset=["sect", "longitude", "latitude"], inplace=True, how="any")

df["longitude"] = df["longitude"].apply(supprimen)
df["latitude"] = df["latitude"].apply(supprimen)

In [13]:
df.shape

(58597, 26)

In [14]:
df.to_csv('base_offres_propre2.csv', encoding='utf8', index=False)

In [ ]:
df = pd.read_csv('base_offres_propre2.csv', header=0, encoding="utf8", dtype={"sect": str} )
df

In [ ]:
import cytoolz as ct

l = list(df.sect_join)
list(ct.unique(l))

In [ ]:
def extract_number(information):
    """Extracts the first number in a string.

        Keyword arguments:
        information -- A string
        """
    try:
        distance = re.findall("\+", information)[0]
        return int(distance)
    except:
        return -1
extract_number("0.554sgk\n")

In [ ]:
def bool_partiel(prop):
    tok = tokenize(prop)
    return ('partiel' in tok)

def bool_plein(prop):
    tok = tokenize(prop)
    return ('plein' in tok)

def quel_temps(prop):

    partiel = bool_partiel(prop)
    plein = bool_plein(prop)

    if (partiel + plein == 1):
        return("partiel"*partiel + "plein"*plein)
    else:
        return "Boloss"

In [ ]:
from geopy import distance
from geopy import Point

def distance(long1, lat1, long2, lat2):
    p1 = Point(long1 + " " + lat1)
    p2 = Point(long2 + " " + lat2)
    return distance.distance(p1,p2).kilometers

In [ ]:
def TF_IDF_matrix(dataframe):

    df["desc_propre"] = df["desc"].apply(lambda w : ' '.join(tokenize(w)))
    countvec = CountVectorizer()
    count = countvec.fit_transform(df.desc_propre)
    vectorizer = TfidfVectorizer(vocabulary=countvec.get_feature_names())
    word_list = countvec.get_feature_names()
    return [pd.DataFrame(vectorizer.fit_transform(df.desc_propre).toarray(), columns=vectorizer.get_feature_names()), word_list]


def frequence_matrix(dataframe):
    df["sect_join"] = df["sect"].apply(lambda w : ' '.join(tokenize2(w)))
    countvec = CountVectorizer()
    return pd.DataFrame(countvec.fit_transform(df.sect_join).toarray(), columns=countvec.get_feature_names())

frequence_matrix(df)


In [ ]:
def frequence_requete(text, word_list):
    tokens = np.array(tokenize(text))
    counts = np.array([np.sum(tokens == e) for e in word_list])
    return counts/len(tokens)

In [ ]:
def score_offre(requete, word_list, tf_idf_offre):
    freq = frequence_requete(requete, word_list)
    
    return np.sum(np.multiply(freq, tf_idf_offre))


In [ ]:
tf_idf_matrix = TF_IDF_matrix(df)[0]

In [ ]:
tf_idf_matrix.shape

In [ ]:
tf_idf_matrix = TF_IDF_matrix(df)[0].as_matrix()
word_list = TF_IDF_matrix(df)[1]



In [ ]:
tf_idf_matrix.to_csv('tf_idf_10000.csv', encoding='utf8', index=False)

In [ ]:
tf_idf_csv = pd.read_csv('tf_idf_10000.csv', header=0, encoding='utf8')

In [ ]:
requete = 'financier'
score_offre(requete, word_list, tf_idf_matrix[0,:])

In [ ]:
np.where((np.array(word_list) == 'automobil') == 1)

In [ ]:
np.sum(tf_idf_csv.as_matrix)

In [ ]:
np.arange(5)

In [ ]:
requete = "je cherche à être mécanicien dans un garage automobile"
scores_offres = [(i, score_offre(requete, word_list, tf_idf_matrix[i,:])) for i in range(nb_offres)]
scores_offres_triees = sorted(scores_offres, key=lambda tup: tup[1], reverse = True)
print(scores_offres_triees[1:10])


In [ ]:
for i in range(10):
    print(df.loc[scores_offres_triees[i][0], "desc"])
    print("\n")